In [ ]:
!pip install transformers -q
!pip install --upgrade accelerate -q
!pip install datasets evaluate -q
!pip install sacrebleu rouge_score jiwer -q
!pip install --upgrade accelerate -q
!pip install wandb -Uqq
!pip install openai

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [4]:
import numpy as np
import pandas as pd
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# ***Create Pipeline***

In [6]:
from google.colab import files

In [8]:
!cp /utils/cf_custom_functions.py /content

In [9]:
import os
import re
import json
import pandas as pd
import cf_custom_functions as cf
pd.set_option('display.max_colwidth', None)

# ***Configuration***

In [10]:
class cfc:

  pred_dir = "/content/drive/MyDrive/predictions/"
  pipeline_var_dir = "/content/drive/MyDrive/Experimente/PIPELINE/VAR2_TrOCR_NLG/"

  trocr_file_path = "/content/drive/MyDrive/predictions/TrOCR_image_to_latex/TrOCR_image-to-latex_with_pad-finetuned.csv"
  metrics_file_path = "/content/drive/MyDrive/Experimente/PIPELINE/VAR2_TrOCR_NLG/metrics_trocr_nlg.json"

### ***Functions***

In [11]:
def remove_punctuation(latex_code):
    pattern = r"(\\,\.|\\,|~|\.|\s)*,?$"
    cleaned_code = re.sub(pattern, '', latex_code, flags=re.MULTILINE)
    return cleaned_code

In [12]:
def pre_processing_OCR(df_ocr:pd.DataFrame)-> pd.DataFrame:

  df = df_ocr.copy()
  clean_preds = []

  for i, row in df.iterrows():
    cl_row = remove_punctuation(row["prediction"])
    clean_preds.append(cl_row)

  df["input_nlg"] = clean_preds
  return df

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')


def tokenize_prediction(prediction:str)-> str:
  input_string = prediction
  pattern = r"\\[a-zA-Z]+|[\w]+|[+\-]|\(|\)|\{|\}|\^|\||\[|\]|<|>|/"
  tokens = re.findall(pattern, input_string)
  result_string = ' '.join(tokens)
  result = result_string.strip()
  return result

In [117]:
def postprocess_latex_predictions(df_preds:pd.DataFrame,df_column:str) -> pd.DataFrame:
  df = df_preds.copy()
  col = df_column
  tokenized_preds = []
  for i, entry in df.iterrows():
    tok_pred = tokenize_prediction(entry[col])
    tokenized_preds.append(tok_pred)

  df["input_nlg"] = tokenized_preds
  return df

In [15]:
def generate_NLG_predictions_from_OCR(test_data:pd.DataFrame, model: object, tokenizer: object,input_column:str,output_column:str,prompt:str)-> pd.DataFrame:

    df = test_data.copy()
    input_col = input_column
    output_col = output_column
    y_preds = []

    for index, row in df.iterrows():
        input = prompt+row[input_col]
        encoded_input = tokenizer(input, truncation=True, return_tensors="pt")
        output = model.generate(**encoded_input, max_length=50)
        decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)
        print(decoded_output)
        y_preds.append(decoded_output)

    y_preds = np.array(y_preds)
    df[output_col] = y_preds

    return df

In [16]:
def post_processing_multi_predictions_OCR(df_preds:pd.DataFrame, pp_column:str, pp_out_column:str)-> pd.DataFrame:

  df = df_preds.copy()
  col = pp_column
  out_col = pp_out_column
  clean_predictions = []

  for i, row in df.iterrows():
    text = row[col]
    # 1. lower case
    text_lower = text.lower()
    # 2. translate -,+ to minus or plus
    translated_text = re.sub(r' - | \+ ', cf.translate_sign, text_lower)
    # 3. translate numbers to text
    cleaned_text = cf.translate_numbers_to_text(translated_text)
    # 4. remove punctations
    cleaned_text = re.sub(r'[^\w\s]', ' ', cleaned_text)
    # 5. remove spaces
    clean_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    clean_predictions.append(clean_text)

  df[out_col] = clean_predictions
  return df

In [17]:
def generate_GPT_predictions(test_data:pd.DataFrame, model: object, tokenizer: object, input_col:str, output_col:str, prompt:str)-> pd.DataFrame:

    df = test_data.copy()
    input_col = input_col
    output_col = output_col
    y_preds = []

    for index, row in df.iterrows():
        input = prompt+row[input_col]
        encoded_input = tokenizer(input, return_tensors="pt")
        output = model.generate(**encoded_input, max_length=50)
        decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)
        gen_text = decoded_output[0].replace(input, "")
        y_preds.append(gen_text)

    y_preds = np.array(y_preds)
    df[output_col] = y_preds

    return df

# ***Load OCR-Predicitons***

In [ ]:
# Load predictions from best ocr model
df_trocr = cf.load_predictions_from_file(cfc.trocr_file_path)

In [ ]:
# Preprocessing and tokenize the predictions
df_input_nlg = postprocess_latex_predictions(df_trocr, "prediction")

In [ ]:
metrics_trocr = cf.compute_OCR_evaluation_metrics(df_input_nlg,"input_nlg")
cf.save_evaluation_metrics("TrOCR",metrics_trocr,cfc.metrics_file_path)

### ***Loading Testset for computing the truth***

In [ ]:
# Load Testset for computing the truth
df_test = pd.read_json("/data/datafile/test_data.json")

# ***Natural Language Generation Models***

## ***1.) Bard-Base***

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint ="facebook/bart-base"
bard_model_dir = "/content/drive/MyDrive/models/NLG_models/Bart-Base_latex-to-text_finetuned-hyper"

bard_model_name = "Bard-Base_OCR_to_text"

bard_tokenizer = AutoTokenizer.from_pretrained(checkpoint)
bard_model = AutoModelForSeq2SeqLM.from_pretrained(bard_model_dir)

In [ ]:
df_preds_bard = generate_NLG_predictions_from_OCR(df_input_nlg, bard_model, bard_tokenizer,"input_nlg","pred_from_ocr","")
df_preds_bard_pp = post_processing_multi_predictions_OCR(df_preds_bard,"pred_from_ocr","clean_pred_from_ocr")
df_preds_bard_pp.to_csv(f"/content/drive/MyDrive/Experimente/PIPELINE/VAR2_TrOCR_NLG/{bard_model_name}_prediction.csv")
metrics_bard = cf.compute_evaluation_metrics(df_preds_bard_pp,"clean_pred_from_ocr")
cf.save_evaluation_metrics(bard_model_name,metrics_bard,cfc.metrics_file_path)

#### ***Test compute the truth***

In [ ]:
df_preds_bard_truth = generate_NLG_predictions_from_OCR(df_test, bard_model, bard_tokenizer,"formula","pred_from_truth","")
df_preds_bard_truth_pp = post_processing_multi_predictions_OCR(df_preds_bard_truth,"pred_from_truth","clean_pred_from_truth")
metrics_bard_truth = cf.compute_evaluation_metrics(df_preds_bard_truth_pp,"clean_pred_from_truth")
cf.save_evaluation_metrics(f"{bard_model_name}-truth",metrics_bard_truth,cfc.metrics_file_path)

# ***2.) T5-Base***

In [139]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

t5_checkpoint = "t5-base"
t5_model_dir = "/content/drive/MyDrive/models/NLG_models/T5-Base-finetuned-latex-to-text-hypertuned"
t5_model_name = "T5-Base_OCR_to_text"

t5_tokenizer = AutoTokenizer.from_pretrained(t5_checkpoint)
t5_model = AutoModelForSeq2SeqLM.from_pretrained(t5_model_dir)

In [ ]:
df_preds_t5 = generate_NLG_predictions_from_OCR(df_input_nlg, t5_model, t5_tokenizer,"input_nlg","pred_from_ocr","translate Latex to Text: ")
df_preds_t5_pp = post_processing_multi_predictions_OCR(df_preds_t5,"pred_from_ocr","clean_pred_from_ocr")
df_preds_t5_pp.to_csv(f"/content/drive/MyDrive/Experimente/PIPELINE/VAR2_TrOCR_NLG/{t5_model_name}_prediction.csv")
metrics_t5 = cf.compute_evaluation_metrics(df_preds_t5_pp,"clean_pred_from_ocr")
cf.save_evaluation_metrics(t5_model_name,metrics_t5,cfc.metrics_file_path)

### ***Check the truth***

In [ ]:
df_preds_t5_truth = generate_NLG_predictions_from_OCR(df_test, t5_model, t5_tokenizer,"formula","pred_from_truth","translate Latex to Text: ")
df_preds_t5_truth_pp = post_processing_multi_predictions_OCR(df_preds_t5_truth,"pred_from_truth","clean_pred_from_truth")
metrics_t5_truth = cf.compute_evaluation_metrics(df_preds_t5_truth_pp,"clean_pred_from_truth")
cf.save_evaluation_metrics(f"{t5_model_name}-truth",metrics_t5_truth,cfc.metrics_file_path)

# ***3.) FLAN-T5-Base***

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

flant5_checkpoint = "google/flan-t5-base"
flant5_model_dir = "/content/drive/MyDrive/models/NLG_models/FLAN-T5-finetuned-latex-to-text_hypertuned"
flant5_model_name = "FLAN-T5-Base_OCR_to_text"

flant5_tokenizer = AutoTokenizer.from_pretrained(flant5_checkpoint)
flant5_model = AutoModelForSeq2SeqLM.from_pretrained(flant5_model_dir)

In [ ]:
df_preds_flant5 = generate_NLG_predictions_from_OCR(df_input_nlg, flant5_model, flant5_tokenizer,"input_nlg","pred_from_ocr","translate Latex to Text: ")
df_preds_flant5_pp = post_processing_multi_predictions_OCR(df_preds_flant5,"pred_from_ocr","clean_pred_from_ocr")
df_preds_flant5_pp.to_csv(f"/content/drive/MyDrive/Experimente/PIPELINE/VAR2_TrOCR_NLG/{flant5_model_name}_prediction.csv")
metrics_flant5 = cf.compute_evaluation_metrics(df_preds_flant5_pp,"clean_pred_from_ocr")
cf.save_evaluation_metrics(flant5_model_name,metrics_flant5,cfc.metrics_file_path)

#### ***Check the truth***

In [ ]:
df_preds_flant5_truth = generate_NLG_predictions_from_OCR(df_test, flant5_model, flant5_tokenizer,"formula","pred_from_truth","translate Latex to Text: ")
df_preds_flant5_truth_pp = post_processing_multi_predictions_OCR(df_preds_flant5_truth,"pred_from_truth","clean_pred_from_truth")
metrics_flant5_truth = cf.compute_evaluation_metrics(df_preds_flant5_truth_pp,"clean_pred_from_truth")
cf.save_evaluation_metrics(f"{flant5_model_name}-truth",metrics_flant5_truth,cfc.metrics_file_path)

# ***4.) ChatGPT***

In [153]:
import openai

# Set up OpenAI API credentials and parameters for GPT-3
openai.api_key = "xxx"
parameters = {"engine": "text-davinci-003", "temperature": 0.2, "max_tokens": 256}

def get_transcription(formula: str) -> list:
    formula_text = formula
    prompt = "Transcript the formula in spoken text:"
    prompt_text = prompt + " " + formula_text
    parameters["prompt"] = prompt_text
    response = openai.Completion.create(**parameters)
    spoken_text = response.choices[0].text.strip().split("\n")
    return spoken_text

In [154]:
def create_transcriptions_OCR(df,input_col, output_col):
  df = df.copy()
  in_col = input_col
  out_col = output_col
  y_preds = []

  for index, row in df.iterrows():
      formula = row[in_col]
      transcription = str(get_transcription(formula))
      print(transcription)
      y_preds.append(transcription)
  df[out_col] = y_preds
  return df

In [ ]:
df_chatgpt = create_transcriptions_OCR(df_input_nlg,"input_nlg","pred_from_ocr")
df_chatgpt_pp = post_processing_multi_predictions_OCR(df_chatgpt,"pred_from_ocr","clean_pred_from_ocr")
df_chatgpt_pp.to_csv(f"/content/drive/MyDrive/Experimente/PIPELINE/VAR2_TrOCR_NLG/ChatGPT_OCR-to-text_prediction.csv")
metrics_chatgpt = cf.compute_evaluation_metrics(df_chatgpt_pp,"clean_pred_from_ocr")
cf.save_evaluation_metrics(f"ChatGPT_OCR-to-text",metrics_chatgpt,cfc.metrics_file_path)

### ***Check the truth***

In [ ]:
df_chatgpt_truth = create_transcriptions_OCR(df_test,"formula","pred_from_ocr")
df_chatgpt_pp_truth = post_processing_multi_predictions_OCR(df_chatgpt_truth,"pred_from_ocr","clean_pred_from_ocr")
metrics_chatgpt_truth = cf.compute_evaluation_metrics(df_chatgpt_pp_truth,"clean_pred_from_ocr")
cf.save_evaluation_metrics(f"ChatGPT_OCR-to-text-truth",metrics_chatgpt_truth,cfc.metrics_file_path)

# ***5.) GPT-3.5 Turbo***

In [161]:
import openai

def get_transcription_gptx(formula,model): #model = "gpt-4", "gpt-3.5-turbo"


  openai.api_key = "xxx"
  response = openai.ChatCompletion.create(
  model=model,
  messages=[
        {"role": "user", "content": "Translate this LaTeX expression to English:"+ formula}],
  temperature = 0.2,
  )
  transcription = response.choices[0].message.content
  return transcription

In [162]:
def generate_transcriptions_with_gptx(df,model,input_col, output_col):
  model = model
  in_col = input_col
  out_col = output_col
  openai.api_key = "xxx"
  df = df.copy()
  y_preds = []
  for index, row in df.iterrows():
    formula = row[in_col]
    transcription = get_transcription_gptx(formula,model)
    print(transcription)
    y_preds.append(transcription)

  y_preds = np.array(y_preds)
  df[out_col] = y_preds
  return df

In [ ]:
df_gpt35t = generate_transcriptions_with_gptx(df_input_nlg,"gpt-3.5-turbo","input_nlg","pred_from_ocr")
df_gpt35t_pp = post_processing_multi_predictions_OCR(df_gpt35t,"pred_from_ocr","clean_pred_from_ocr")
df_gpt35t_pp.to_csv(f"/content/drive/MyDrive/Experimente/PIPELINE/VAR2_TrOCR_NLG/GPT-35T_OCR-to-text_prediction.csv")
metrics_gpt35t = cf.compute_evaluation_metrics(df_gpt35t_pp,"clean_pred_from_ocr")
cf.save_evaluation_metrics(f"GPT-35T_OCR-to-text",metrics_gpt35t,cfc.metrics_file_path)

### ***Checking the truth***

In [ ]:
df_gpt35t_truth = generate_transcriptions_with_gptx(df_test,"gpt-3.5-turbo","formula","pred_from_ocr")
df_gpt35t_pp_truth = post_processing_multi_predictions_OCR(df_gpt35t_truth,"pred_from_ocr","clean_pred_from_ocr")
metrics_gpt35t_truth = cf.compute_evaluation_metrics(df_gpt35t_pp_truth,"clean_pred_from_ocr")
cf.save_evaluation_metrics(f"GPT-35T_OCR-to-text-truth",metrics_gpt35t_truth,cfc.metrics_file_path)

# ***6.) GPT-4***

In [ ]:
df_gpt4 = generate_transcriptions_with_gptx(df_input_nlg,"gpt-4","input_nlg","pred_from_ocr")
df_gpt4_pp = post_processing_multi_predictions_OCR(df_gpt4,"pred_from_ocr","clean_pred_from_ocr")
df_gpt4_pp.to_csv(f"/content/drive/MyDrive/Experimente/PIPELINE/VAR2_TrOCR_NLG/GPT-4_OCR-to-text_prediction.csv")
metrics_gpt4 = cf.compute_evaluation_metrics(df_gpt4_pp,"clean_pred_from_ocr")
cf.save_evaluation_metrics(f"GPT-4_OCR-to-text",metrics_gpt4,cfc.metrics_file_path)

### ***Checking the truth***

In [ ]:
df_gpt4_truth = generate_transcriptions_with_gptx(df_test,"gpt-4","formula","pred_from_ocr")
df_gpt4_pp_truth = post_processing_multi_predictions_OCR(df_gpt4_truth,"pred_from_ocr","clean_pred_from_ocr")
metrics_gpt4_truth = cf.compute_evaluation_metrics(df_gpt4_pp_truth,"clean_pred_from_ocr")
cf.save_evaluation_metrics(f"GPT-4_OCR-to-text-truth",metrics_gpt4_truth,cfc.metrics_file_path)

# ***7.) GPT-2***

In [ ]:
from transformers import AutoModelWithLMHead

gpt2_checkpoint = "gpt2-medium"
gpt2_model_dir = "/content/drive/MyDrive/models/NLG_models/GPT-2-Medium-latex-to-text-hypertuned"
gpt2_model_name = "GPT2-Medium_OCR_to_text"

gpt2_tokenizer = AutoTokenizer.from_pretrained(gpt2_checkpoint)
gpt2_model = AutoModelWithLMHead.from_pretrained(gpt2_model_dir)

In [ ]:
df_preds_gpt2 = generate_GPT_predictions(df_input_nlg, gpt2_model, gpt2_tokenizer,"input_nlg","pred_from_ocr","generate spoken text of: ")
df_preds_gpt2_pp = post_processing_multi_predictions_OCR(df_preds_gpt2,"pred_from_ocr","clean_pred_from_ocr")
df_preds_gpt2_pp.to_csv(f"/content/drive/MyDrive/Experimente/PIPELINE/VAR2_TrOCR_NLG/{gpt2_model_name}_prediction.csv")
metrics_gpt2 = cf.compute_evaluation_metrics(df_preds_gpt2_pp,"clean_pred_from_ocr")
cf.save_evaluation_metrics(gpt2_model_name,metrics_gpt2,cfc.metrics_file_path)

# ***Summary***

In [179]:
df_nlg1  = pd.read_json("/content/drive/MyDrive/Experimente/PIPELINE/VAR2_TrOCR_NLG/metrics_trocr_nlg.json")
df_nlg1 = df_nlg1.transpose()
df_nlg1 = df_nlg1[["BLEU","TER","TER-ACC","ROUGE-1","ROUGE-2","ROUGE-L"]]
df_nlg1["BLEU"] = pd.to_numeric(df_nlg1["BLEU"])
df_nlg1["TER"] = pd.to_numeric(df_nlg1["TER"])
df_nlg1["TER-ACC"] = pd.to_numeric(df_nlg1["TER-ACC"])
df_nlg1["ROUGE-1"] = pd.to_numeric(df_nlg1["ROUGE-1"])
df_nlg1["ROUGE-2"] = pd.to_numeric(df_nlg1["ROUGE-2"])
df_nlg1["ROUGE-L"] = pd.to_numeric(df_nlg1["ROUGE-L"])
#df_nlg["WER"] = pd.to_numeric(df_nlg["WER"])
df_nlg1 = df_nlg1[["BLEU", "TER", "TER-ACC", "ROUGE-1","ROUGE-2","ROUGE-L"]].sort_values(by="BLEU", ascending=False)
formatted_df = df_nlg1.style.format({"BLEU": "{:.2f}", "TER": "{:.2f}", "TER-ACC": "{:.2f}", "ROUGE-1": "{:.2f}", "ROUGE-2": "{:.2f}","ROUGE-L": "{:.2f}"})

formatted_df.set_table_styles([
    {'selector': 'tr', 'props': [('text-align', 'left')]},
    {'selector': 'td', 'props': [('text-align', 'right')]}
])

display(formatted_df)

,BLEU,TER,TER-ACC,ROUGE-1,ROUGE-2,ROUGE-L
TrOCR,65.74,27.71,72.29,84.16,67.18,83.44
Bard-Base_OCR_to_text,59.28,39.29,60.71,74.50,55.34,71.67
T5-Base_OCR_to_text,57.30,40.06,59.94,72.76,53.90,69.98
FLAN-T5-Base_OCR_to_text,54.47,40.97,59.03,72.62,52.66,69.38
ChatGPT_OCR-to-text,53.01,48.05,51.95,73.33,54.15,68.95
GPT-4_OCR-to-text,43.39,58.97,41.03,71.06,50.69,67.83
GPT2-Medium_OCR_to_text,41.62,63.87,36.13,56.00,38.08,51.49
GPT-35T_OCR-to-text,34.72,77.53,22.47,62.94,41.98,58.42


In [ ]:
df_nlg1_latex = df_nlg1.to_latex(caption='VAR2 Experiment Pipeline')
print(df_nlg1_latex)